In [1]:
from keras.layers import Conv2D, UpSampling2D, Dropout, Activation, InputLayer,Dense
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import tensorflow as tf

In [2]:
# Loading images
X = []
i = 0
for filename in os.listdir('Train/'):
        X.append(img_to_array(load_img('Train/'+filename)))
X = np.array(X, dtype=float)

In [3]:
# Splitting train and test sets
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128

In [4]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 5 # 10 was
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [5]:
# # Model Structure

# model = Sequential()
# model.add(InputLayer(input_shape=(256, 256, 1)))
# model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
# model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
# model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
# model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(UpSampling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(UpSampling2D((2, 2)))
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(Conv2D(2, (3, 3), activation='relu', padding='same'))
# model.add(UpSampling2D((2, 2)))

# model.compile(optimizer='adam',loss='mse', metrics=['accuracy'])

In [6]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))


model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [7]:
# Train model      
%time model.fit(image_a_b_gen(batch_size), epochs=10,steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 12s 121ms/step - loss: 0.0162 - accuracy: 0.9579
Epoch 2/10
100/100 [==============================] - 12s 124ms/step - loss: 0.0011 - accuracy: 0.9743
Epoch 3/10
100/100 [==============================] - 12s 120ms/step - loss: 7.8518e-04 - accuracy: 0.9741
Epoch 4/10
100/100 [==============================] - 12s 120ms/step - loss: 5.2557e-04 - accuracy: 0.9744
Epoch 5/10
100/100 [==============================] - 12s 121ms/step - loss: 4.2956e-04 - accuracy: 0.9747
Epoch 6/10
100/100 [==============================] - 12s 121ms/step - loss: 3.6445e-04 - accuracy: 0.9751
Epoch 7/10
100/100 [==============================] - 12s 121ms/step - loss: 3.0968e-04 - accuracy: 0.9752
Epoch 8/10
100/100 [==============================] - 12s 120ms/step - loss: 2.7121e-04 - accuracy: 0.9757
Epoch 9/10
100/100 [==============================] - 12s 123ms/step - loss: 2.4282e-04 - accuracy: 0.9760
Epoch 10/10
100/100 [========================

In [8]:
model.save('mew.h5')

In [9]:
# model = load_model('model.h5')

In [10]:
# Evaluation
print("Loss:\t\t\tAccuracy:")
print(model.evaluate(Xtest, Ytest, verbose=1, batch_size=batch_size))

Loss:			Accuracy:
1/1 [==============================] - 0s 3ms/step - loss: 2.6990e-04 - accuracy: 0.9701
[0.00026989809703081846, 0.9701080322265625]


In [11]:
# Batch image testing

color_me = []
for filename in os.listdir('Test/'):
        color_me.append(img_to_array(load_img('Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))
output = model.predict(color_me)
output = output * 128
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

In [12]:
# Single image Testing

color_me = load_img('show.jpg')
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,0]
color_me = color_me.reshape((1,)+color_me.shape+(1,))
output = model.predict(color_me)
output = output * 128
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("img_result.png", lab2rgb(cur))
    imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
